# sveltejs/sveltekit

> 

In [ ]:
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    llm_process_page,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
source = GitSource(
    name="sveltejs/sveltekit",
    doc_dir="apps/svelte.dev/content/docs/kit",
    repo="https://github.com/sveltejs/svelte.dev",
    commit="main",
    ecosystems=["svelte"])

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

#### Read the markdown files

In [ ]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)
# tree.model_dump()

#### Process the tree

In [ ]:
tree.children[1].children[-2]

DocItem(origPath=PosixPath('20-core-concepts/60-remote-functions.md'), name='60-remote-functions.md', displayName='60-remote-functions.md', digest='', short_digest='', essence='', relevant=True, usage=Usage(input=0, output=0, details=None), token_counts=TokenCounts(fulltext=0, digest=0, short_digest=0), fulltext='---\nNOTE: do not edit this file, it is generated in apps/svelte.dev/scripts/sync-docs/index.ts\ntitle: Remote functions\n---\n\n<blockquote class="since note">\n\t<p>Available since 2.27</p>\n</blockquote>\n\nRemote functions are a tool for type-safe communication between client and server. They can be _called_ anywhere in your app, but always _run_ on the server, meaning they can safely access [server-only modules](server-only-modules) containing things like environment variables and database clients.\n\nCombined with Svelte\'s experimental support for [`await`](/docs/svelte/await-expressions), it allows you to load and manipulate data directly inside your components.\n\nThi

In [ ]:
# res = await llm_process_page(settings, tree.children[1].children[-2], "sveltejs/sveltekit")

In [ ]:
# from IPython.display import Markdown

# display(Markdown(res.digest))

In [ ]:
# tree.subdirs = tree.subdirs[:2] # For testing process just the first 2 secstion (intro and runes)

In [ ]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+0.864s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing 10-getting-started/10-introduction.md
+0.301s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing 10-getting-started/20-creating-a-project.md
+0.016s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing 10-getting-started/25-project-types.md
+0.016s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing 10-getting-started/30-project-structure.md
+0.019s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing 10-getting-started/40-web-standards.md
+0.015s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing 10-getting-started/index.md
+0.015s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing 20-core-concepts/10-routing.md
+0.019s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing 20-core-concepts/20-load.md
+0.015s DEBUG preprocessor/nbs/lovely_docs/doc

#### Calculate cost

In [ ]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 347,944
  Output tokens: 114,973
  Total tokens: 462,917
Cost:
  Total: $0.92
  Input: $0.35
  Output: $0.57


#### save the results

In [ ]:
save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)